# Funding

A set of tables for handling funding activities


In [66]:
#| default_exp Funding

In [67]:
#| export
#| hide
from biobib.table import Table, sh, safe_float
from biobib.sheet import Sheet
from nbdev import show_doc
import pandas as pd
import time
import copy

In [68]:
#| hide
from fastcore.test import *
from fastcore.test import *
from fastcore.basics import *
from fastcore.foundation import *

### Define a template for funding activities

Here is a template we can use for funding activities. It generates a longtable which can span multiple pages, and provides the years, funding source, proposal name and amount for each grant

In [69]:
funding_biobib_template = r'''% UC Bio-bib Funding Table
% Created on \VAR{created}

\begin{longtable}{p{1.75cm}>{\raggedright}p{2.cm}p{5cm}|p{1.25cm}p{1.25cm}p{1.25cm}|p{0.75cm}p{0.75cm}}
Years & Agency/Source & Title & Total Amount & Total to UCSB & Personal Share & Role & New/Cont.\\
\hline 
\endfirsthead

\multicolumn{8}{c}%
{{\VAR{table_name} - continued from previous page }} \\ \\
Years & Agency/Source & Title & Total Amount & Total to UCSB & Personal Share & Role & New/Cont.\\
\hline 
\endhead

\\
\multicolumn{8}{c}%
{{ \VAR{table_name} continued on next page }} \\
\endfoot

\hline \hline
\endlastfoot

\BLOCK{for grant in items}
\VAR{grant|make_years} & \VAR{grant.Source|tex_escape} & \VAR{grant.Title|tex_escape} & \VAR{grant['Total Amount']|tex_escape} & \VAR{grant['Total to UCSB']|tex_escape} & \BLOCK{if grant['Pooled Funds']} Pooled \BLOCK{else} \VAR{grant['Personal Share']|tex_escape} \BLOCK{endif} & \VAR{grant.Role|tex_escape}  & \VAR{grant.Type|tex_escape} \\ 
\BLOCK{endfor}
& & \multicolumn{1}{r} {\bf Total New :} & \VAR{total_new['Total Amount']|tex_escape} & \VAR{total_new['Total to UCSB']|tex_escape} & \VAR{total_new['Total Personal Share']|tex_escape} & & \\
& & \multicolumn{1}{r} {\bf Total :} & \VAR{total['Total Amount']|tex_escape} & \VAR{total['Total to UCSB']|tex_escape} & \VAR{total['Total Personal Share']|tex_escape} & & \\

\end{longtable}

'''

In [70]:
class Funding(Table):
    """ 
    A Funding class of Table for use in generating biobibs and CVs 
    """
    def __init__(
            self,
            sheet:Sheet=None, # Google sheet object
            worksheet:str='Grants',   # worksheet name in google sheet
            csv_file:str=None, # optional csv file (if using csv files) 
            table_name:str=None, # Will be determined from category.
            cumulative:bool=False, # Is this table cumulative? 
            template=funding_biobib_template # template content
    ): 
        filters = {
            'make_years': self.make_years
        }
        super().__init__(
            sheet=sheet,
            worksheet=worksheet, 
            table_name=table_name,
            csv_file=csv_file,
            filters=filters, 
            template=template)
        self.cumulative = cumulative
        self.df = self.clean_df()
    
    def total(self, new=False):
        if new:
            funds = self.df[self.df['Type'] == 'New']
        else:
            funds = self.df

        return {
            'Total Amount': '${:,.0f}'.format(
                funds['Total Amount'].replace('[\$,]', '', regex=True).apply(safe_float).sum()),
            'Total to UCSB': '${:,.0f}'.format(
                funds['Total to UCSB'].replace('[\$,]', '', regex=True).apply(safe_float).sum()),
            'Total Personal Share': '${:,.0f}'.format(
                funds['Personal Share'].replace('[\$,]', '', regex=True).apply(safe_float).sum())
        }


    def clean_df(self):
        df = copy.deepcopy(self.df)
        df = super().table_clean_df()
        df = df[df['Funded'].astype(int) == 1]
        # Replace NaN with a 'nan' string for checking later
        df['Start Date'].fillna('nan', inplace=True)
        df['End Date'].fillna('nan', inplace=True)
        df = df.sort_values(by=['Start Year', 'Total Amount'],
                            ascending=[True, False])
        return df

    def make_years(self, row):
        if row['Start Date'] != 'nan' and row['End Date'] != 'nan':
            return "{start}-{end}".format(
                start=row['Start Date'],
                end=row['End Date'])
        else:
            return "{start}-{end}".format(
                start=row['Start Year'],
                end=row['End Year'])

    def render_template(self):
        rendered_tex = self.template.render(
            table_name=self.table_name,
            created=time.strftime("%Y-%m-%d %H:%M"),
            items=list(self.df.to_dict('records')),
            total=self.total(new=False),
            total_new=self.total(new=True)
        )
        return rendered_tex

    

### Provide a data cleaning function for the service table



In [71]:
funding_table = Funding(
    sheet=sh,worksheet='Grants',table_name='Funding',
    template=funding_biobib_template)

In [72]:
test_eq(funding_table.table_name,'Funding')
test_eq(list(funding_table.df.columns),
        ['Start Year', 'End Year', 'Year', 'Start Date', 
         'End Date', 'Title', 'Source', 'Program', 
         'Total Amount', 'Total to UCSB', 'Personal Share', 
         'Role', 'Summer Salary', 'Pooled Funds', 'Award Number', 
         'Duration', 'Yearly Amount', 'NSF', 'Federal, Non-NSF', 
         'Foundation', 'Other', 'Eval', 
         'Type', 'Funded', 'Active'])

In [73]:
print(funding_table.render_template())

% UC Bio-bib Funding Table
% Created on 2023-06-29 18:35

\begin{longtable}{p{1.75cm}>{\raggedright}p{2.cm}p{5cm}|p{1.25cm}p{1.25cm}p{1.25cm}|p{0.75cm}p{0.75cm}}
Years & Agency/Source & Title & Total Amount & Total to UCSB & Personal Share & Role & New/Cont.\\
\hline 
\endfirsthead

\multicolumn{8}{c}%
{{Funding - continued from previous page }} \\ \\
Years & Agency/Source & Title & Total Amount & Total to UCSB & Personal Share & Role & New/Cont.\\
\hline 
\endhead

\\
\multicolumn{8}{c}%
{{ Funding continued on next page }} \\
\endfoot

\hline \hline
\endlastfoot

7/1/2022-6/30/2025 & Zegar Foundation & Ecohydrological Resilience of Vegetation to Water Availability and Drought & \$672,079 & \$672,079 &  Pooled  & PI  & New \\ 
1/1/2023-12/31/2023 & National Science Foundation & NSF Convergence Accelerator Track J: Predicting the effect of climate extremes on the food system to improve resilience of global and local food security & \$748,674 & \$748,674 &  Pooled  & Co-PI  & New \\ 
9/

In [74]:
funding_cv_template = r'''% CV Funding Table
% Created on \VAR{created}

{\bf Total: }\VAR{total['Total Amount']|tex_escape}  \\

\begin{longtable}{p{1.75cm}>{\raggedright}p{2.cm}p{12cm}}
\endfirsthead

\multicolumn{3}{c}%
{{\VAR{table_name} - continued from previous page }} \\ \\
\endhead

\\
\multicolumn{3}{c}%
{{ \VAR{table_name} continued on next page }} \\
\endfoot


\endlastfoot


\BLOCK{for grant in items|reversed }
\VAR{grant['Start Year']}-\VAR{grant['End Year']} & \VAR{grant.Source|tex_escape} & \VAR{grant.Title|tex_escape}, \VAR{grant.Role|tex_escape} (\VAR{grant['Total Amount']|tex_escape}) \\ 
\BLOCK{endfor}

\end{longtable}
'''

In [75]:
cv_table = Funding(
    sheet=sh,worksheet='Grants',table_name='Funding',
    cumulative=True,
    template=funding_cv_template)

In [76]:
print(cv_table.render_template())

% CV Funding Table
% Created on 2023-06-29 18:35

{\bf Total: }\$23,711,262  \\

\begin{longtable}{p{1.75cm}>{\raggedright}p{2.cm}p{12cm}}
\endfirsthead

\multicolumn{3}{c}%
{{Funding - continued from previous page }} \\ \\
\endhead

\\
\multicolumn{3}{c}%
{{ Funding continued on next page }} \\
\endfoot


\endlastfoot


2023-2028 & Department of Defense & The Climate-Food-Urbanization Nexus and the Precursors of Instability in Africa, Co-PI (\$3,073,150) \\ 
2023-2025 & Department of Energy & Improving ESS approaches to evapotranspiration partitioning through data fusion, Co-PI (\$399,007) \\ 
2023-2023 & National Science Foundation & NSF Convergence Accelerator Track J: Predicting the effect of climate extremes on the food system to improve resilience of global and local food security, Co-PI (\$748,674) \\ 
2022-2025 & Zegar Foundation & Ecohydrological Resilience of Vegetation to Water Availability and Drought, PI (\$672,079) \\ 
2019-2023 & National Science Foundation & CNH2-L:  Li

In [77]:
cv_table.write_template(path='../tex/CV/')

In [78]:
#| hide
import nbdev; nbdev.nbdev_export()